# 🚀 SpectraMind V50 — Quickstart Notebook

Fast path to validate your **environment → configs → pipeline** for the **NeurIPS 2025 Ariel Data Challenge**.

This notebook mirrors our **CLI-first, Hydra-driven, reproducibility-focused** workflow (Typer CLI + Hydra configs + DVC) so you can smoke-test the stack in minutes without re-implementing any logic.

⚠️ **Note:** All steps default to `--fast`, `--dry-run`, or sample subsets. Switch to full runs only after these checks pass.


## 0. Runtime Helper — Resolve `spectramind` Launcher

Some environments expose the CLI differently. This helper resolves in order:
1. `spectramind` (on PATH)
2. `poetry run spectramind`
3. `python -m spectramind`

Provides `sm(cmd)` and `sm_print(cmd)` helpers.

In [ ]:
import os, shlex, shutil, subprocess, sys

def _resolve_spectramind_cmd():
    if shutil.which("spectramind"):
        return ["spectramind"]
    if shutil.which("poetry"):
        try:
            out = subprocess.run(["poetry", "run", "spectramind", "--version"],
                                 stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
            if out.returncode == 0:
                return ["poetry", "run", "spectramind"]
        except Exception:
            pass
    return [sys.executable, "-m", "spectramind"]

SM = _resolve_spectramind_cmd()
print("Resolved spectramind launcher:", " ".join(shlex.quote(p) for p in SM))

def sm(cmd: str, check=False, capture=False):
    args = shlex.split(cmd)
    res = subprocess.run(SM + args,
                         check=check,
                         stdout=subprocess.PIPE if capture else None,
                         stderr=subprocess.STDOUT if capture else None,
                         text=True)
    return res

def sm_print(cmd: str):
    print("$", " ".join(SM + shlex.split(cmd)))


## 1. Repo Sanity Checks
Validate core tooling (Python, Poetry, Git, DVC) and GPU snapshot.

In [ ]:
!python --version
!pip --version
!poetry --version || echo '⚠️ Poetry not found'
!git --version
!dvc --version || echo '⚠️ DVC not found'

try:
    import torch
    print("PyTorch:", torch.__version__)
    if torch.cuda.is_available():
        print("CUDA ✓ — device:", torch.cuda.get_device_name(0))
    else:
        print("CUDA not available (CPU)")
except Exception:
    print("PyTorch not installed (ok for quickstart)")


## 2. CLI Smoke Test

In [ ]:
sm_print("--version")
out = sm("--version", capture=True)
print(out.stdout)

sm_print("--help")
out = sm("--help", capture=True)
print("\n".join(out.stdout.splitlines()[:20]))


## 3. Hydra Config Composition

In [ ]:
cmd = "train model=airs_gnn optimizer=adamw training.fast_dev_run=true"
sm_print(cmd)
sm(cmd)


## 4. Mini End-to-End Pipeline

In [ ]:
for cmd in [
    "test --fast",
    "calibrate --sample 3 --fast",
    "train --epochs 1 training.fast_dev_run=true",
    "diagnose dashboard --no-umap --no-tsne --outdir outputs/diagnostics_quick",
    "submit --dry-run"
]:
    sm_print(cmd)
    sm(cmd)


## 5. Cheat Sheet — Common Workflows
```bash
spectramind train model=fgs1_mamba optimizer=adamw training.epochs=50
spectramind calibrate --sample 10
spectramind submit --config configs/config_v50.yaml
```
Logs land in `logs/v50_debug_log.md`.

## 6. DVC Data Tips

In [ ]:
%%bash
set -euo pipefail
if command -v dvc >/dev/null 2>&1; then
  echo "DVC detected — pulling latest data..."
  dvc pull || echo '⚠️ non-fatal'
  echo "Optionally recompute: dvc repro"
else
  echo "DVC not installed — skipping"
fi


## 7. Python Helper — Run CLI Inside Notebook

In [ ]:
res = sm("--version", capture=True)
print(res.stdout)

res = sm("train training.fast_dev_run=true", capture=True)
for line in (res.stdout or "").splitlines():
    if "loss" in line.lower() or "gll" in line.lower():
        print(line)


## 8. Where to Look Afterwards
- Outputs: `outputs/`
- Logs: `logs/v50_debug_log.md`
- Diagnostics HTML: from `diagnose dashboard`
- Configs: `configs/`
